<H1 align="center">Dashboards Visualizing<br><br>Hundreds of Millions of Datapoints<br><br>in 30 Lines of Python</H1>
<br>
<H3 align="center">Philipp Rudiger, James A. Bednar, and Jean-Luc Stevens<br><br>Continuum Analytics<H3>


**Let's say you want to:** 

* Make it easy to explore some dataset with custom widgets.

**But then you have to:**
* Spend days of effort to get something working in a notebook
* Build an opaque mishmash of domain-specific, widget, and plotting code
* Start over from scratch whenever you need to:
    - Deploy in a standalone server
    - Visualize different aspects of your data
    - Scale up to larger (>100K) datasets

Here we'll show a simple, flexible, powerful, step-by-step way to solve problems like this, by combining open-source libraries:

* [**Dask**](http://dask.pydata.org): Efficient out-of-core/distributed computation on massive datasets
* [**Fastparquet**](https://fastparquet.readthedocs.io): Efficient storage for columnar data
* [**HoloViews**](http://holoviews.org): Declarative objects for instantly visualizable data
* [**GeoViews**](http://geo.holoviews.org): Easy mix-and-matching of geographic data with custom plots
* [**Bokeh**](http://bokeh.pydata.org): Interactive plotting in web browsers, controlled by Python
* [**Numba**](http://numba.pydata.org): Accelerated machine code for inner loops
* [**Datashader**](https://github.com/bokeh/datashader): Rasterizing huge datasets quickly using Dask and Numba
* [**Param**](https://github.com/ioam/param): Declaring user-relevant parameters in domain-specific code

We'll be working through this process:
-  Step 1: Get some data
-  Step 2: Prototype a plot in a notebook
-  Step 3: Declare your Parameters
-  Step 4: Get a widget-based UI for free
-  Step 5: Link your Parameters to your data
-  Step 6: Widgets now control your interactive plots
-  Step 7: Deploy your dashboard

In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb, parambokeh
import pandas as pd
import dask.dataframe as dd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, PlotSize

## Step 1: Get some data

* Here we'll use a subset of the often-studied NYC Taxi dataset
* About 12 million points of GPS locations from taxis
* Stored in the efficient Parquet format for easy access
* Loaded into a Dask dataframe for multi-core<br>(and if needed, out of core or distributed) computation

In [ ]:
df = dd.read_parquet('./data/nyc_taxi.parq/').persist()
print(len(df))
df.head(2)

## Step 2: Prototype a plot in a notebook

* A text-based representation isn't very useful for big datasets like this, so we need to build a plot
* But we don't want to start a software project, so we use HoloViews:
    - Simple, declarative way to annotate your data for visualization
    - Large library of Elements with associated visual representation
    - Elements combine (lay out or overlay) easily
* And we'll want live interactivity, so we'll use a Bokeh backend
* But our data is much too big for Bokeh directly, so we'll use Datashader to rasterize it first

In [ ]:
hv.extension('bokeh')
points = hv.Points(df, kdims=['pickup_x', 'pickup_y'], vdims=['passenger_count'])
options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
taxi_trips

Let's put the data in context, overlaying it on a map:

In [ ]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/'
                                   'World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))
tiles * taxi_trips

## Step 3: Declare your Parameters

Now that we've prototyped a nice plot, we want it to be widely sharable, with controls for safe and easy exploration. 

So the next step: declare what the intended user can change, with:

  - type and range checking
  - documentation strings
  - default values
  
The Param library allows declaring Python attributes having these features<br>(and more, such as dynamic values and inheritance).

## NYC Taxi Parameters

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    plot        = param.ObjectSelector(default="pickup", objects=["pickup","dropoff"])
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    passengers  = param.Range(default=(1, 9), bounds=(1, 9), doc="""
        Filter for taxi trips by number of passengers""")

Each Parameter is a normal Python attribute, but with special checks and functions run automatically when getting or setting.

Parameters capture your goals and your knowledge about your domain, declaratively.

### Class level parameters

In [ ]:
NYCTaxiExplorer.alpha

In [ ]:
NYCTaxiExplorer.alpha = 0.5
NYCTaxiExplorer.alpha

### Validation

In [ ]:
try:
   NYCTaxiExplorer.alpha = '0'
except Exception as e:
    print(e)

### Instance parameters

In [ ]:
explorer = NYCTaxiExplorer(alpha=0.5)
explorer.alpha

In [ ]:
NYCTaxiExplorer.alpha

## Step 4: Get a widget-based UI for free

* Parameters are purely declarative, but contain all the information needed to build interactive widgets
* ParamNB generates UIs from Parameters, using ipywidgets

In [ ]:
paramnb.Widgets(NYCTaxiExplorer)

* ipywidgets work with Jupyter Dashboards Server for deployment

* Declaration of parameters is independent of the UI library used
* ParamBokeh generates UIs from Parameters, using Bokeh widgets

In [ ]:
parambokeh.Widgets(NYCTaxiExplorer)

* Bokeh widgets work with Bokeh Server for deployment

## Step 5: Link your Parameters to your data

Because the Parameters defined earlier are *about* a plot, it makes sense to combine the parameter and plotting declarations into a single object:

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.Range(default=(1, 9), bounds=(1, 9))

    def make_view(self, x_range, y_range, alpha, colormap, plot, passengers):
        map_tiles = tiles.opts(style=dict(alpha=alpha), plot=options) 

        points = hv.Points(df, kdims=[plot+'_x', plot+'_y'], vdims=['passenger_count'])
        selected = points.select(passenger_count=passengers)
        taxi_trips = datashade(selected, x_sampling=1, y_sampling=1, cmap=colormap,
                               dynamic=False, x_range=x_range, y_range=y_range,
                               width=800, height=475)
        return map_tiles * taxi_trips

Note that the `NYCTaxiExplorer` class is entirely declarative (no widgets), and can be used "by hand" to provide range-checked and type-checked plotting:

In [ ]:
explorer = NYCTaxiExplorer(alpha=0.2,plot="dropoff")
explorer.make_view(x_range=None,y_range=None,alpha=explorer.alpha, colormap=explorer.colormap, 
                   plot=explorer.plot, passengers=explorer.passengers)

## Step 6: Widgets now control your interactive plots

In [ ]:
explorer = NYCTaxiExplorer()
paramnb.Widgets(explorer, callback=explorer.event)
hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])

## Step 7: Deploy your dashboard

If you want to share your work with people who don't use Python, you'll now want to run a server with this same code.

* If you used **ParamBokeh**, deploy with **Bokeh Server**:
    - Write the above code to a file ``nyc_parambokeh.py``,<br> saving the `Widgets` object as the attribute `doc`
    - ``bokeh serve nyc_parambokeh.py``

* If you used **ParamNB**, deploy with **Jupyter Dashboard Server**:
    - Use Jupyter Dashboards Extension to select cells from the notebook to display
    - Use preview mode to see layout
    - Use Jupyter Dashboards Server to deploy
    - Note various caveats below

# Branching out

Much more ambitious apps are possible with very little additional code or effort:

* Adding additional linked or separate subplots of any type; see [holoviews.org](http://holoviews.org)
* Declaring code that runs for clicking or selecting *within* the Bokeh plot; see "streams" at [holoviews.org](http://holoviews.org)
* Using multiple sets of widgets of many different types; see [ParamNB](https://github.com/ioam/paramnb) and [ParamBokeh](https://github.com/ioam/parambokeh)
* Using datasets too big for any one machine, with [Dask.Distributed](https://distributed.readthedocs.io)

# Future work

* Jupyter Dashboards Server not currently maintained; requires older ipywidgets version
* Bokeh Server is mature and well supported, but does not currently support drag-and-drop layout like Jupyter Dashboards does
* ParamBokeh still needs some polishing and work to make it ready for widespread use; ParamNB is more mature so far
* Let us know what you would like to see out of these tools!